# Data Cleaning
In this notebook, I'll look at the data and deal with missing values. Also excluding dataset with data quality deemed suspicious based on the original document (https://github.com/stanford-policylab/opp/blob/master/data_readme.md)

In [1]:
import pandas as pd
import numpy as np

pd.set_option('precision', 4)
pd.options.display.max_seq_items = None
pd.options.display.max_columns = 50

In [2]:
df0 = pd.read_csv('DATA/full_df.csv', index_col = 0, dtype = object)

In [3]:
#df.columns

In [4]:
# lower case columns to make it searchable
df0.columns = [x.lower() for x in df0.columns]

## Raw Data
Raw data are original data. I'll separate these out since we may not need to use it.

In [5]:
RAW = df0[df0.columns[df0.columns.str.startswith('raw')]]

In [6]:
df = df0.drop(RAW[1:], axis = 1)

## Missing values & Subsetting Data
There are many missing values. I'll use outcome as a target variable and drop missing outcome (should not infer target variable).

In [7]:
df.outcome.isnull().sum()

103262

In [8]:
df.outcome.value_counts()

citation    193921
warning      96064
arrest       10431
summons       2580
Name: outcome, dtype: int64

In [9]:
df = df.dropna(subset = ['outcome'])

Some columnns are just breakdown of outcome. I'll remove those.

In [10]:
df = df.drop(['citation_issued', 'warning_issued', 'arrest_made'], axis = 1)

Some columns are post-outcome occurrences, I'll remove those.

In [11]:
df = df.drop(['reason_for_arrest'], axis = 1)

Hardfort, CT dataset was considered 'suspicious' by others who have made the cleaned individual dataset from raw data. I'll remove those.

In [12]:
df = df.loc[df.fname != 'ct_hartford_2020_04_01.csv',:]

Remove a column if it's missing majority of the data.

In [13]:
df = df.drop(df.columns[df.isnull().sum() > (len(df)/2)], axis = 1)

Remove the entire location if the location misses majority of race and sex information.

In [14]:
tmp = df.groupby('fname').agg({
                        'subject_race': lambda x: x.isnull().sum()/len(x), 
                        'subject_sex': lambda x: x.isnull().sum()/len(x)})

In [15]:
to_remove = list(tmp[(tmp.subject_race > 0.5) | (tmp.subject_sex > 0.5)].index)

In [16]:
df = df[~df.fname.isin(to_remove)]

Remove all info with missing race and sex information (key predictor without means to infer value)

In [17]:
df = df.dropna(subset = ['subject_race', 'subject_sex'])

For the purpose of this analysis, I'll drop rows if subject age and violation is missing.

In [18]:
df = df.dropna(subset = ['subject_age', 'violation'])

Fill missing time with average hour

In [19]:
# fill time with average hour
hour = str(int(df.time.dropna().apply(lambda x: int(x[:2])).mean()))
df.time = df.time.fillna(f'{hour}:00:00')

Get city/state from fname and remove location.

In [20]:
df['state'] = [x.split('_')[0] for x in df.fname]
df['city'] = [x[:-15][3:] for x in df.fname]

In [21]:
df = df.drop('location', axis = 1)

In [22]:
df = df.drop('fname', axis = 1)

For missing lat/long, impute the general location from the city info.

In [23]:
from geopy.geocoders import Nominatim

def get_lat_long(location):
    geolocator = Nominatim(user_agent = "open_polici_dat")
    location = geolocator.geocode(location)
    return (location.latitude, location.longitude)    

In [24]:
df['location'] = np.where(df['city'] == 'statewide', df['state'], df['city'] + ', ' + df['state'])

In [33]:
locations = set(df.location)
all_loc = dict.fromkeys(locations, ())

for loc in locations:
    try:
        all_loc[loc] = get_lat_long(loc)
    except AttributeError:
        print(f'error: {loc}')
        

error: oklahoma_city, ok


In [34]:
all_loc['oklahoma_city, ok'] = get_lat_long('Oklahoma City, OK')

In [35]:
df['lat'] = df.location.apply(lambda x: all_loc[x][0])
df['lng'] = df.location.apply(lambda x: all_loc[x][1])

For search conducted, I'll add unknown as category for missing values.

In [36]:
df.search_conducted = df.search_conducted.fillna('unknown')

For types, if I find vehicular related words from violation I'll put vehicular, otherwise I'll put 'others'.

In [37]:
df['violation'] = df.violation.apply(lambda x: x.lower())

In [38]:
vehic_list_ = ['veh', 'ticket', 'traf', 'dmv', 'belt', 'speed', 'stop sign', 'dui', 
         'registration', 'parking', 'driv', 'yield', 'insurance', 'right of way', 'oper', 'fast',
              'bicycle', 'phone', 'lane', 'helmet', 'oneway', 'lighting', 'stop/stand', 'reg', 'backing',
               'highway', 'light', 'over height', 'freeway', 'license', 'hydrant', 'turn', 'safety chain']
df['vehicular_indicator'] = df.violation.apply(lambda x: any(vehic_list_ in x for vehic_list_ in vehic_list_))

In [39]:
df['type'] = np.where((df.type.isnull()) & (df.vehicular_indicator), 'vehicular', df.type)

In [40]:
ped_list = ['pedest', 'public', 'in park', 'walk', 'larceny', 'tresp', 'shoplift', 'curfew',
            'gathering', 'disorderly conduct', 'street', 'closed park', 'touching', 'theft', 'outdoors']
df['pedestrian_indicator'] = df.violation.apply(lambda x: any(ped_list in x for ped_list in ped_list))
df['type'] = np.where((df.type.isnull()) & (df.pedestrian_indicator), 'pedestrian', df.type)

In [41]:
df.type = df['type'].fillna('others')

### Data Type
Let's check the final data types to find any abnormalities.

In [42]:
#df.info()

Age shouldn't be an object. Let's check the values.

In [43]:
#df.subject_age.unique()
df.subject_age = df.subject_age.astype('int64')

### Violations
From violation texts, I'll try to identify indication for different violation types and add as columns.

In [83]:
import string

# removing special characters and digits
#trans = str.maketrans('', '', string.punctuation+string.digits)
exclist = string.punctuation + string.digits
trans = str.maketrans(exclist, ' '*len(exclist))


df['violation_split'] = [x.translate(trans).split() for x in df.violation]

In [84]:
# first, figuring out frequent words
from itertools import chain
all_texts = list(chain(*df['violation_split']))

In [85]:
from collections import Counter
freq_words = dict(Counter(all_texts).most_common())

In [93]:
# check spellings
from textblob import TextBlob 
for text in freq_words.keys():
    corrected = 
[(text, str(TextBlob(text).correct())) for text in freq_words.keys() if TextBlob(text).correct() != text] 

[('mph', TextBlob("may")),
 ('nrs', TextBlob("mrs")),
 ('muni', TextBlob("mundi")),
 ('radar', TextBlob("rear")),
 ('oper', TextBlob("over")),
 ('sp', TextBlob("s")),
 ('req', TextBlob("red")),
 ('ftc', TextBlob("etc")),
 ('ins', TextBlob("in")),
 ('dl', TextBlob("do")),
 ('lt', TextBlob("it")),
 ('cam', TextBlob("can")),
 ('veh', TextBlob("ve")),
 ('susp', TextBlob("sup")),
 ('vni', TextBlob("vi")),
 ('vt', TextBlob("it")),
 ('unregistered', TextBlob("registered")),
 ('sl', TextBlob("s")),
 ('dui', TextBlob("due")),
 ('aa', TextBlob("a")),
 ('reg', TextBlob("red")),
 ('viol', TextBlob("vol")),
 ('evid', TextBlob("evil")),
 ('fincial', TextBlob("final")),
 ('resp', TextBlob("rest")),
 ('zz', TextBlob("ze")),
 ('revok', TextBlob("revolt")),
 ('tabs', TextBlob("tabes")),
 ('traf', TextBlob("trap")),
 ('unsafe', TextBlob("safe")),
 ('lic', TextBlob("lie")),
 ('poss', TextBlob("pass")),
 ('exp', TextBlob("ex")),
 ('mv', TextBlob("my")),
 ('revocation', TextBlob("revolution")),
 ('mobl', Te

### Export
Exporting the cleaned dataframe.

In [265]:
#df.to_pickle('PKL/clean_df.pkl')